In [1]:
import pandas as pd
import numpy as np
import duckdb

In [2]:
data = pd.read_csv('poverty_raw_data.csv')
data


,record_id,region_code,country_name,country_code,survey_acronym,reporting_year,reporting_level,survey_coverage,survey_year,welfare_type,...,decile9,decile10,cpi,ppp,reporting_pop,reporting_gdp,reporting_pce,is_interpolated,distribution_type,estimation_type
0,1,NaN,Angola,AGO,HBS,2000,national,national,2000.21,consumption,...,0.158687,0.402455,0.014200,107.269989,16394062,1840.353828,NaN,0,micro,survey
1,2,SSA,Angola,AGO,IBEP-MICS,2008,national,national,2008.50,consumption,...,0.162333,0.323117,0.303449,107.269989,21691522,3061.636272,1216.312411,0,micro,survey
2,3,SSA,Angola,AGO,IDREA,2018,national,national,2018.17,consumption,...,0.159390,0.396244,1.231024,107.269989,31273533,2677.249833,1548.153535,0,micro,survey
3,4,ECA,Armenia,ARM,ILCS,2012,national,national,2012.00,consumption,...,0.141960,0.245402,0.888864,167.312210,2914421,3278.670451,2953.453311,0,micro,survey
4,5,ECA,Albania,ALB,EWS,1996,national,national,1996.00,consumption,...,0.148995,0.212156,0.399635,50.357372,3168033,1633.552004,1716.336806,0,micro,survey
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2313,2314,MNA,"Yemen, Rep.",YEM,HBS,2014,national,national,2014.00,consumption,...,0.152674,0.294356,0.518205,255.683585,27753304,2123.935040,NaN,0,micro,survey
2314,2315,SSA,South Africa,ZAF,KIDS,1993,national,national,1993.00,consumption,...,0.176722,0.466597,0.250919,6.549363,42525440,4269.700164,2415.898997,0,group,survey
2315,2316,SSA,South Africa,ZAF,HIES,2000,national,national,2000.75,consumption,...,0.177200,0.449305,0.427681,6.549363,46813266,4735.665532,2811.527872,0,group,survey
2316,2317,SSA,South Africa,ZAF,IES,2005,national,national,2005.00,consumption,...,0.167912,0.542298,0.523158,6.549363,49017147,5458.230166,3324.357678,0,micro,survey


In [3]:
print(duckdb.query('''SELECT DISTINCT country_name, survey_acronym, survey_year 
                   FROM data 
                   WHERE region_code = 'LAC' 
                        AND survey_coverage = 'national' '''))


┌───────────────┬────────────────┬─────────────┐
│ country_name  │ survey_acronym │ survey_year │
│    varchar    │    varchar     │   double    │
├───────────────┼────────────────┼─────────────┤
│ Bolivia       │ EH             │      2014.0 │
│ Bolivia       │ EH             │      2015.0 │
│ Bolivia       │ EH             │      2021.0 │
│ Brazil        │ PNAD           │      1986.0 │
│ Brazil        │ PNAD           │      1989.0 │
│ Brazil        │ PNADC-E1       │      2019.0 │
│ Brazil        │ PNADC-E5       │      2020.0 │
│ Belize        │ LFS            │      1993.0 │
│ Brazil        │ PNAD           │      1996.0 │
│ Brazil        │ PNAD           │      2001.0 │
│   ·           │  ·             │         ·   │
│   ·           │  ·             │         ·   │
│   ·           │  ·             │         ·   │
│ El Salvador   │ EHPM           │      2012.0 │
│ Uruguay       │ ECH            │      2008.0 │
│ Uruguay       │ ECH            │      2009.0 │
│ Uruguay       │ EC

In [20]:
print(duckdb.query('''WITH AvgPceCountries AS (
                   SELECT country_name, AVG(reporting_pce) as avg_pce
                   FROM data 
                   WHERE region_code = 'SSA'
                   GROUP BY country_name
)
                   
                   SELECT country_name, avg_pce
                   FROM AvgPceCountries 
                   WHERE avg_pce = (SELECT MIN(avg_pce) FROM AvgPceCountries) '''))

┌──────────────────┬──────────────┐
│   country_name   │   avg_pce    │
│     varchar      │    double    │
├──────────────────┼──────────────┤
│ Congo, Dem. Rep. │ 305.26031975 │
└──────────────────┴──────────────┘

